In [19]:
# %load_ext autoreload
# %autoreload 2

In [20]:
# !pip install -q "numpy<2"
# !pip install -q albumentations
# !pip install -q ultralytics

In [21]:
from urllib.request import urlretrieve; urlretrieve("https://raw.githubusercontent.com/c0z0c/jupyter_hangul/refs/heads/beta/helper_utils.py", "helper_utils.py")
import importlib
import helper_utils as hu
importlib.reload(hu)

🌐 https://c0z0c.github.io/jupyter_hangul
ℹ️ NumPy 2.0.2 (v2.x+): 호환성 모드 적용됨
Mounted at /content/drive
✅ 설정 완료: 한글 폰트, plt 전역 등록, pandas 확장, 캐시 기능
pd commit 저장 경로 = /content/drive/MyDrive


<module 'helper_utils' from '/content/helper_utils.py'>

In [22]:
# 기본 라이브러리

# --- Scikit-learn: 데이터 전처리, 모델, 평가 ---
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import (
    fetch_california_housing, load_iris, make_moons, make_circles,
    load_breast_cancer, load_wine
)
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import average_precision_score

# --- 기타 라이브러리 ---
from PIL import Image
from PIL import ImageFilter
from PIL import ImageDraw
import albumentations as A
import IPython.display
#from tqdm import tqdm
from tqdm.notebook import tqdm

# --- PyTorch: 딥러닝 관련 ---
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2
from torchvision.datasets import CocoDetection
from torchvision.transforms import functional as TF
from torch.nn import CrossEntropyLoss
from collections import OrderedDict

# --- 기타 ---
import re
import os
import sys
import json
import math
import random
import yaml
import shutil
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
from datetime import datetime
from datetime import timezone, timedelta
import pytz
__kst = pytz.timezone('Asia/Seoul')

# GPU 설정
__device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
__device_cpu = torch.device('cpu')

  # 재현 가능한 결과를 위해
np.random.seed(42)
torch.manual_seed(42)
if __device == 'cuda':
    torch.cuda.manual_seed_all(42)

print(f"라이브러리 로드 완료 사용장치:{__device}")

라이브러리 로드 완료 사용장치:cuda


In [23]:
def create_dataset(n_examples):
    """모든 가능한 월 이름을 포함한 포괄적인 데이터셋 생성"""
    dataset = []

    # 모든 월 이름과 축약형을 명시적으로 포함
    full_months = ['January', 'February', 'March', 'April', 'May', 'June',
                   'July', 'August', 'September', 'October', 'November', 'December']

    short_months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    for i in range(n_examples):
        year = random.randint(1990, 2030)
        month = (i % 12) + 1
        day = random.randint(1, 28)

        current_date = datetime(year, month, day)

        # 포맷 선택을 더 균등하게
        format_choice = i % 4

        if format_choice == 0:  # 전체 월 이름
            input_str = f"{full_months[month-1]} {day:02d}, {year}"
        elif format_choice == 1:  # 축약 월 이름
            input_str = f"{day:02d} {short_months[month-1]} {year}"
        elif format_choice == 2:  # 숫자 슬래시
            input_str = f"{year}/{month:02d}/{day:02d}"
        else:  # 숫자 하이픈
            input_str = f"{month:02d}-{day:02d}-{year}"

        output_str = current_date.strftime('%Y-%m-%d')
        dataset.append((input_str, output_str))

    random.shuffle(dataset)
    return dataset

In [24]:
raw_data = create_dataset(5000)
raw_data[:10]

[('04-20-2024', '2024-04-20'),
 ('08-14-2002', '2002-08-14'),
 ('January 19, 2018', '2018-01-19'),
 ('2011/07/22', '2011-07-22'),
 ('10 Oct 2000', '2000-10-10'),
 ('27 Feb 1992', '1992-02-27'),
 ('04-23-2012', '2012-04-23'),
 ('2007/11/08', '2007-11-08'),
 ('15 Jun 2017', '2017-06-15'),
 ('20 Feb 1993', '1993-02-20')]

In [25]:
# 문자 단위 토큰화를 위한 새로운 Lang 클래스
class CharLang:
    def __init__(self, name):
        self.name = name

        # PAD, SOS, EOS 명시적으로 추가 (PAD=0)
        self.char2index = {"PAD": 0, "SOS": 1, "EOS": 2}
        self.index2char = {0: "PAD", 1: "SOS", 2: "EOS"}
        self.n_chars = 3

    def add_sentence(self, sentence):
        for char in sentence:
            if char not in self.char2index:
                self.char2index[char] = self.n_chars
                self.index2char[self.n_chars] = char
                self.n_chars += 1

# 문자 단위 언어 클래스 생성
date_input_lang = CharLang("date_input")
date_output_lang = CharLang("date_output")

# 문자 단위로 vocabulary 구축
for pair in raw_data:
    date_input_lang.add_sentence(pair[0])
    date_output_lang.add_sentence(pair[1])


In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np

# 데이터 및 CharLang 클래스는 이미 정의되어 사용 가능하다고 가정
# date_input_lang, date_output_lang 인스턴스 사용

In [27]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, emb_dim, n_heads, pf_dim, dropout):
        super().__init__()
        self.attn = nn.MultiheadAttention(emb_dim, n_heads, dropout=dropout, batch_first=True)
        self.ff = nn.Sequential(
            nn.Linear(emb_dim, pf_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(pf_dim, emb_dim)
        )
        self.norm1 = nn.LayerNorm(emb_dim)
        self.norm2 = nn.LayerNorm(emb_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_mask=None):
        # Multi-Head Attention
        _src = self.norm1(src)
        attn_output, _ = self.attn(_src, _src, _src, attn_mask=src_mask)
        src = src + self.dropout(attn_output) # Add & Norm 1

        # Feed Forward
        _src = self.norm2(src)
        ff_output = self.ff(_src)
        src = src + self.dropout(ff_output) # Add & Norm 2

        return src

class TransformerDecoderLayer(nn.Module):
    def __init__(self, emb_dim, n_heads, pf_dim, dropout):
        super().__init__()
        # 1. Masked Multi-Head Attention (Self-Attention)
        self.self_attn = nn.MultiheadAttention(emb_dim, n_heads, dropout=dropout, batch_first=True)
        # 2. Encoder-Decoder Attention (Cross-Attention)
        self.enc_attn = nn.MultiheadAttention(emb_dim, n_heads, dropout=dropout, batch_first=True)
        # 3. Feed Forward
        self.ff = nn.Sequential(
            nn.Linear(emb_dim, pf_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(pf_dim, emb_dim)
        )

        self.norm1 = nn.LayerNorm(emb_dim)
        self.norm2 = nn.LayerNorm(emb_dim)
        self.norm3 = nn.LayerNorm(emb_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, trg, enc_src, trg_mask, src_mask):
        # 1. Masked Multi-Head Attention (Self-Attention)
        _trg = self.norm1(trg)
        self_attn_output, _ = self.self_attn(_trg, _trg, _trg, attn_mask=trg_mask)
        trg = trg + self.dropout(self_attn_output) # Add & Norm 1

        # 2. Encoder-Decoder Attention (Cross-Attention)
        _trg = self.norm2(trg)
        enc_attn_output, _ = self.enc_attn(_trg, enc_src, enc_src, attn_mask=src_mask)
        trg = trg + self.dropout(enc_attn_output) # Add & Norm 2

        # 3. Feed Forward
        _trg = self.norm3(trg)
        ff_output = self.ff(_trg)
        trg = trg + self.dropout(ff_output) # Add & Norm 3

        return trg

In [28]:
import torch.nn as nn
import torch

class Seq2SeqTransformer(nn.Module):
    def __init__(self, input_dim, output_dim, emb_dim, n_heads, pf_dim, n_layers, dropout, max_len):
        super().__init__()

        # 1. nn.Sequential 대신 nn.ModuleList 사용 (마스크 인수를 전달하기 위함)
        self.encoder = nn.ModuleList([TransformerEncoderLayer(emb_dim, n_heads, pf_dim, dropout) for _ in range(n_layers)])
        self.decoder = nn.ModuleList([TransformerDecoderLayer(emb_dim, n_heads, pf_dim, dropout) for _ in range(n_layers)])

        self.input_emb = nn.Embedding(input_dim, emb_dim)
        self.output_emb = nn.Embedding(output_dim, emb_dim)

        # 위치 인코딩
        self.pos_emb = nn.Embedding(max_len, emb_dim)

        self.fc_out = nn.Linear(emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # device 저장

    def forward(self, src, trg, src_mask, trg_mask):
        # src: [배치 크기, src_len], trg: [배치 크기, trg_len]
        batch_size = src.shape[0]

        # 1. 입력 임베딩 및 위치 인코딩

        # 인코더 입력 처리
        src_len = src.shape[1]
        # 💡 수정: src_len에 맞는 위치 텐서 생성 (pos 변수 정의)
        src_pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        src = self.dropout(self.input_emb(src) + self.pos_emb(src_pos)) # [배치 크기, src_len, emb_dim]

        # 디코더 입력 처리
        trg_len = trg.shape[1]
        # 💡 수정: trg_len에 맞는 위치 텐서 생성
        trg_pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        trg = self.dropout(self.output_emb(trg) + self.pos_emb(trg_pos)) # [배치 크기, trg_len, emb_dim]

        # 2. 인코더 실행
        enc_src = src
        for layer in self.encoder:
             # 명시적으로 src_mask를 각 인코더 레이어에 전달
            enc_src = layer(enc_src, src_mask)

        # 3. 디코더 실행
        output = trg
        for layer in self.decoder:
             # 명시적으로 trg_mask와 src_mask를 각 디코더 레이어에 전달
            output = layer(output, enc_src, trg_mask, src_mask)

        # 4. 최종 출력
        output = self.fc_out(output) # [배치 크기, trg_len, output_dim]
        return output

In [29]:
def make_trg_mask(trg):
    # trg: [배치 크기, 타겟 시퀀스 길이]
    trg_len = trg.shape[1]

    # 삼각형 마스크 (Causal Mask) 생성
    # 0은 어텐션 허용, -inf는 어텐션 차단
    trg_mask = torch.tril(torch.ones((trg_len, trg_len), dtype=torch.bool)).to(trg.device)
    trg_mask = trg_mask.float().masked_fill(trg_mask == 0, float('-inf')).masked_fill(trg_mask == 1, float(0.0))
    # trg_mask: [타겟 시퀀스 길이, 타겟 시퀀스 길이]
    return trg_mask

In [30]:
# 문자열을 인덱스 리스트로 변환
def indexes_from_sentence(lang, sentence):
    return [lang.char2index[char] for char in sentence]

# 페어를 텐서로 변환
def tensor_from_pair(pair, input_lang, output_lang, max_input_len, max_output_len):
    input_indexes = indexes_from_sentence(input_lang, pair[0])
    output_indexes = indexes_from_sentence(output_lang, pair[1])

    # SOS, EOS 토큰 추가
    input_tensor = torch.tensor([input_lang.char2index["SOS"]] + input_indexes + [input_lang.char2index["EOS"]], dtype=torch.long)
    output_tensor = torch.tensor([output_lang.char2index["SOS"]] + output_indexes + [output_lang.char2index["EOS"]], dtype=torch.long)

    # 패딩 (PAD 인덱스로 채우기)
    pad_idx_in = input_lang.char2index["PAD"]
    pad_idx_out = output_lang.char2index["PAD"]

    input_pad = torch.full((max_input_len,), pad_idx_in, dtype=torch.long)
    input_pad[:len(input_tensor)] = input_tensor

    output_pad = torch.full((max_output_len,), pad_idx_out, dtype=torch.long)
    output_pad[:len(output_tensor)] = output_tensor

    return input_pad, output_pad

In [31]:
# 하이퍼파라미터 설정
INPUT_DIM = date_input_lang.n_chars
OUTPUT_DIM = date_output_lang.n_chars
EMB_DIM = 256
N_HEADS = 8
PF_DIM = 512
N_LAYERS = 2 # MVP이므로 2개 층 사용
DROPOUT = 0.1
MAX_INPUT_LEN = 20 # 최대 입력 길이 (데이터에 따라 설정)
MAX_OUTPUT_LEN = 12 # 최대 출력 길이 (데이터에 따라 설정: YYYY-MM-DD + SOS/EOS = 12)
N_EPOCHS = 1000 # 충분한 학습 반복
BATCH_SIZE = len(raw_data) # 작은 데이터셋이므로 전체 배치 사용

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model = Seq2SeqTransformer(INPUT_DIM, OUTPUT_DIM, EMB_DIM, N_HEADS, PF_DIM, N_LAYERS, DROPOUT, max(MAX_INPUT_LEN, MAX_OUTPUT_LEN)).to(device)
# optimizer = optim.Adam(model.parameters(), lr=0.0005)
# criterion = nn.CrossEntropyLoss(ignore_index=0) # 패딩(인덱스 0)은 손실 계산에서 제외

model = Seq2SeqTransformer(INPUT_DIM, OUTPUT_DIM, EMB_DIM, N_HEADS, PF_DIM, N_LAYERS, DROPOUT, max(MAX_INPUT_LEN, MAX_OUTPUT_LEN)).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0005)
# 변경: PAD 인덱스를 가져와서 ignore_index로 사용
pad_idx = date_output_lang.char2index["PAD"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

print(f"모델 파라미터 수: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

모델 파라미터 수: 2,656,526


In [32]:
# 데이터 준비
input_tensors, output_tensors = zip(*[tensor_from_pair(p, date_input_lang, date_output_lang, MAX_INPUT_LEN, MAX_OUTPUT_LEN) for p in raw_data])
SRC = torch.stack(input_tensors).to(device)
TRG = torch.stack(output_tensors).to(device)

pbar = tqdm(range(N_EPOCHS))
for epoch in pbar:
    model.train()
    optimizer.zero_grad()

    # 인코더 입력 마스크 (패딩 마스크) - 이 예시에서는 패딩 마스크를 간단하게 생략하거나 None 처리 가능
    # 하지만 실제로는 패딩 토큰 인덱스(0)에 맞춰 src_mask를 만들어야 함. 여기서는 None으로 간주하고 생략
    src_mask = None

    # 디코더 입력 (타겟 시퀀스에서 마지막 EOS 토큰 제외)
    trg_input = TRG[:, :-1]

    # 인과 관계 마스크 생성
    trg_mask = make_trg_mask(trg_input)

    # 모델 예측
    #output = model(SRC, trg_input, src_mask, None) # src_mask는 None으로 단순화
    output = model(SRC, trg_input, src_mask, trg_mask)

    # 실제 타겟 (SOS 토큰 제외)
    output_dim = output.shape[-1]
    output = output.contiguous().view(-1, output_dim)
    trg_target = TRG[:, 1:].contiguous().view(-1)

    # 손실 계산 및 역전파
    loss = criterion(output, trg_target)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch: {epoch+1:03} | Train Loss: {loss.item():.4f}')

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch: 100 | Train Loss: 0.0004
Epoch: 200 | Train Loss: 0.0002
Epoch: 300 | Train Loss: 0.0001
Epoch: 400 | Train Loss: 0.0000
Epoch: 500 | Train Loss: 0.0001
Epoch: 600 | Train Loss: 0.0000
Epoch: 700 | Train Loss: 0.0000
Epoch: 800 | Train Loss: 0.0000
Epoch: 900 | Train Loss: 0.0000
Epoch: 1000 | Train Loss: 0.0000


In [34]:
def translate_sentence(model, src_tensor, input_lang, output_lang, max_len, device):
    model.eval()
    with torch.no_grad():
        # 인코딩 시작
        src = src_tensor.unsqueeze(0).to(device)
        src_mask = None # 패딩 마스크 생략

        src_len = src.shape[1]

        # 💡 수정 1: src_pos 텐서 정의 (device 지정)
        src_pos = torch.arange(0, src_len, device=device).unsqueeze(0)
        src = model.dropout(model.input_emb(src) + model.pos_emb(src_pos))

        # 💡 수정 2: model.encoder를 반복문으로 실행
        enc_src = src
        for layer in model.encoder:
            enc_src = layer(enc_src, src_mask)

        # 디코딩 (자동 회귀)
        trg_indexes = [output_lang.char2index["SOS"]]

        for i in range(max_len):
            trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)
            trg_mask = make_trg_mask(trg_tensor)

            trg_len = trg_tensor.shape[1]

            # 💡 수정 3: trg_pos 텐서 정의 (device 지정)
            trg_pos = torch.arange(0, trg_len, device=device).unsqueeze(0)
            trg_emb = model.dropout(model.output_emb(trg_tensor) + model.pos_emb(trg_pos))

            # 💡 수정 4: model.decoder를 반복문으로 실행
            output = trg_emb
            for layer in model.decoder:
                 # 디코더의 각 레이어를 명시적으로 호출
                output = layer(output, enc_src, trg_mask, src_mask)

            # 마지막 토큰에 대한 예측만 사용
            pred_token = model.fc_out(output[:, -1, :]).argmax(dim=-1).item()
            trg_indexes.append(pred_token)

            if pred_token == output_lang.char2index["EOS"]:
                break

        trg_tokens = [output_lang.index2char[i] for i in trg_indexes]
        return ''.join(trg_tokens[1:-1])

# 정확도 테스트
correct_count = 0
for i, pair in enumerate(raw_data):
    src_tensor = SRC[i]
    predicted_output = translate_sentence(model, src_tensor, date_input_lang, date_output_lang, MAX_OUTPUT_LEN, device)

    if predicted_output == pair[1]:
        correct_count += 1

    print(f"\r입력: {pair[0]:<20} | 정답: {pair[1]:<10} | 예측: {predicted_output:<10} | {'O' if predicted_output == pair[1] else 'X'}", end='')

accuracy = correct_count / len(raw_data)
print(f"\n최종 트랜스포머 MVP 정확도: {accuracy * 100:.2f}%")

입력: 21 Oct 2020          | 정답: 2020-10-21 | 예측: 2020-10-21 | O
최종 트랜스포머 MVP 정확도: 100.00%
